In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Cargar los datos desde el archivo Excel
file_path = r'C:\Users\christian_luna\Documents\Maestria\TopicosDeIndustria1\datos_trafico_guadalajara.xlsx'
df = pd.read_excel(file_path)

# Inicializar la aplicación de Dash
app = dash.Dash(__name__)

# Opciones de filtros
zonas_options = [{'label': zona, 'value': zona} for zona in df['Zona'].unique()]
vehiculos_options = [{'label': vehiculo, 'value': vehiculo} for vehiculo in df['Tipo de Vehículo'].unique()]

# Diseño del dashboard
app.layout = html.Div([
    html.H1("Dashboard de Tráfico - Zona Metropolitana de Guadalajara"),
    
    # Menú de filtros
    html.Div([
        html.Label('Seleccionar Zona:'),
        dcc.Dropdown(id='zona-dropdown', options=zonas_options, value=zonas_options[0]['value']),
        
        html.Label('Seleccionar Tipo de Vehículo:'),
        dcc.Dropdown(id='vehiculo-dropdown', options=vehiculos_options, value=vehiculos_options[0]['value']),
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    # Gráficos
    dcc.Graph(id='mapa-calor'),
    dcc.Graph(id='histograma-velocidad')
])

# Callback para actualizar el mapa de calor y el histograma según los filtros seleccionados
@app.callback(
    [Output('mapa-calor', 'figure'),
     Output('histograma-velocidad', 'figure')],
    [Input('zona-dropdown', 'value'),
     Input('vehiculo-dropdown', 'value')]
)
def actualizar_graficos(zona, vehiculo):
    # Filtrar el conjunto de datos según la zona y el tipo de vehículo seleccionado
    df_filtrado = df[(df['Zona'] == zona) & (df['Tipo de Vehículo'] == vehiculo)]
    
    # Mapa de calor: Congestión en función del tiempo
    fig_mapa_calor = px.density_heatmap(df_filtrado, x='Fecha y Hora', y='Zona',
                                        z='Número de Vehículos',
                                        title=f'Congestión en {zona} para {vehiculo}',
                                        labels={'z': 'Número de Vehículos'})
    
    # Histograma de velocidades
    fig_histograma = px.histogram(df_filtrado, x='Velocidad Promedio (km/h)',
                                  title=f'Histograma de Velocidades para {zona} y {vehiculo}',
                                  labels={'x': 'Velocidad Promedio (km/h)', 'y': 'Frecuencia'})

    return fig_mapa_calor, fig_histograma

# Correr la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)
